In [ ]:
from google3.experimental.users.jballe.colab.jax import *
from collections import defaultdict
import math
from typing import Callable
import flax


In [ ]:
from jax import config
# config.update("jax_debug_nans", True)
# config.update("jax_debug_infs", True)


In [ ]:
jax.devices()


In [ ]:
%no_vertical_scrollbar

# brick wall
# reference = mediapy.read_image("https://unsplash.com/photos/9f3tCfHoGDE/download?ixid=M3wxMjA3fDB8MXx0b3BpY3x8aVVJc25WdGpCMFl8fHx8fDJ8fDE3MTQ1OTMxNjR8&force=true&w=600")
# grass
# reference = mediapy.read_image("https://unsplash.com/photos/C2PCa6DhlYE/download?ixid=M3wxMjA3fDB8MXxhbGx8fHx8fHx8fHwxNzEzODc3OTg1fA&force=true&w=400")
# zion
# reference = mediapy.read_image("https://unsplash.com/photos/LvbsrFcbDo8/download?ixid=M3wxMjA3fDB8MXxhbGx8NTJ8fHx8fHwyfHwxNzE0Njc4MjA5fA&force=true&w=600")
# manhattanhenge
# reference = mediapy.read_image("https://unsplash.com/photos/8gWXf45We4Q/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MXx8bWFuaGF0dGFuaGVuZ2V8ZW58MHx8fHwxNzE0Njc5MjUzfDA&force=true&w=400")
# flowers
# reference = mediapy.read_image("https://unsplash.com/photos/1kGvoC9g2DY/download?ixid=M3wxMjA3fDB8MXxhbGx8fHx8fHx8fHwxNzE0NjgwOTU3fA&force=true&w=400")
# boat
# reference = mediapy.read_image("https://unsplash.com/photos/kmcYs7FUFpA/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MTB8fG9jZWFuJTIwYm9hdHxlbnwwfHx8fDE3MTQ2ODQ1NTJ8MA&force=true&w=600")
# old town
# reference = mediapy.read_image("https://unsplash.com/photos/-uAtopUxcgU/download?ixid=M3wxMjA3fDB8MXx0b3BpY3x8RnpvM3p1T0hONnd8fHx8fDJ8fDE3MTQ2ODQ5OTN8&force=true&w=400")
# alhambra
# reference = mediapy.read_image("https://unsplash.com/photos/nowbpvuHT9k/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MjIzfHxhbGhhbWJyYXxlbnwwfHx8fDE3MTQ2ODg4MTl8MA&force=true&w=400")
# snake
reference = mediapy.read_image("https://unsplash.com/photos/gUeEQ-eOhzs/download?ixid=M3wxMjA3fDB8MXxhbGx8Mzc2fHx8fHx8Mnx8MTcxNDY5ODM3OXw&force=true&w=600")
# room
# reference = mediapy.read_image("https://unsplash.com/photos/RRjC0r8GwpI/download?ixid=M3wxMjA3fDB8MXxhbGx8NjA1fHx8fHx8Mnx8MTcxNDY5ODg4NHw&force=true&w=600")
# beach people
# reference = mediapy.read_image("https://unsplash.com/photos/GI1TjAX3MtI/download?ixid=M3wxMjA3fDB8MXxhbGx8Njg4fHx8fHx8Mnx8MTcxNDY5OTM3Nnw&force=true&w=400")
# beach rock
# reference = mediapy.read_image("https://unsplash.com/photos/elyAcITxBYI/download?ixid=M3wxMjA3fDB8MXxhbGx8NzA1fHx8fHx8Mnx8MTcxNDY5OTM3OHw&force=true&w=400")

reference = (reference / 255).transpose((2, 0, 1))
print(reference.shape)

mediapy.show_image(reference.transpose((1, 2, 0)))


In [ ]:
LP_KERNEL = np.array([.25, .5, .25])

plt.plot(scipy.fft.fftfreq(1000), abs(scipy.fft.fft(LP_KERNEL, 1000)))
plt.xticks([-.5, -.25, 0, .25, .5])
plt.ylim(ymin=0)
plt.grid()


In [ ]:
LP_KERNEL = jnp.array([.25, .5, .25])
LP_KERNEL = jnp.outer(LP_KERNEL, LP_KERNEL)[None, None]


def lowpass(features, stride):
  """Lowpass filters a feature array of shape (batch, height, width)."""
  return jax.lax.conv(
      features[:, None],
      LP_KERNEL,
      window_strides=(stride, stride),
      padding="same",
  ).squeeze(1)


def compute_multiscale_stats(features, levels):
  """Computes local mean and variance of a feature array."""
  squared = jnp.square(features)
  means = []
  variances = []
  for _ in range(levels):
    m = lowpass(features, 1)
    p = lowpass(squared, 1)
    means.append(m)
    variances.append(p - jnp.square(m))
    features = m[..., ::2, ::2]
    squared = p[..., ::2, ::2]
  return means, variances


def wasserstein_distortion(features_a, features_b, log_sigma, levels=5):
  means_a, variances_a = compute_multiscale_stats(features_a, levels)
  means_b, variances_b = compute_multiscale_stats(features_b, levels)
  wd_maps = [jnp.square(features_a - features_b)]
  assert len(means_a) == len(means_b) == len(variances_a) == len(variances_b)
  for ma, mb, va, vb in zip(means_a, means_b, variances_a, variances_b):
    sa = jnp.sqrt(va + 1e-4)
    sb = jnp.sqrt(vb + 1e-4)
    wd_maps.append(jnp.square(ma - mb) + jnp.square(sa - sb))
  assert len(wd_maps) == levels + 1
  dist = 0.
  stuff = defaultdict(list, wd_maps=wd_maps)
  for i, wd_map in enumerate(wd_maps):
    weight = jax.nn.relu(1 - abs(log_sigma - i))
    stuff["weights"].append(weight)
    dist += jnp.mean(weight * wd_map)
    if i > 0:
      log_sigma = lowpass(log_sigma[None], 2).squeeze(0)
  return dist, stuff


In [ ]:
VGG_SHIFT = (1.0 + np.array([-.030, -.088, -.188])) / 2.0
VGG_SCALE = np.array([.458, .448, .450]) / 2.0


class VggBlock(flax.linen.Module):
  """One block within the VGG network."""
  num_features: int
  num_layers: int

  @flax.linen.compact
  def __call__(self, x):
    for _ in range(self.num_layers):
      x = flax.linen.Conv(
          features=self.num_features,
          kernel_size=(3, 3),
          padding="same",
      )(x)
      x = flax.linen.relu(x)
    return x


class VggNet(flax.linen.Module):
  """VGG network which returns intermediate activations."""

  @flax.linen.compact
  def __call__(self, x):
    assert x.shape[-2] >= 16, str(x.shape)
    assert x.shape[-3] >= 16, str(x.shape)
    outputs = [x]
    for params in ((64, 2), (128, 2), (256, 3), (512, 3), (512, 3)):
      x = VggBlock(*params)(x)
      outputs.append(x)
      x = flax.linen.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    return outputs


def read_params(filename=None):
  params = VggNet().init(jax.random.PRNGKey(0), jnp.zeros([1, 224, 224, 3]))
  if filename is None:
    filename = "/cns/tp-d/home/theis/vgg_params.bin"
  with gfile.Open(filename, "rb") as f:
    return flax.serialization.from_bytes(params, f.read())


def compute_features(params, image):
  image = jnp.transpose(image, (1, 2, 0))[None]
  image = (image - VGG_SHIFT) / VGG_SCALE
  features = VggNet().apply(params, image)
  return [jnp.squeeze(f, 0).transpose((2, 0, 1)) for f in features]


In [ ]:
sigma = np.outer(
    scipy.signal.gaussian(reference.shape[-2], 100),
    scipy.signal.gaussian(reference.shape[-1], 100),
)
sigma /= sigma.max()
sigma = (1 - sigma) * 32
log_sigma = jax.nn.relu(jnp.log2(sigma))
log_sigma = jnp.full(log_sigma.shape, 5.)

plt.imshow(log_sigma)
plt.colorbar()


In [ ]:
params = read_params()
reference_features = compute_features(params, reference)

print([f.shape for f in reference_features])


In [ ]:
f = reference_features[1][-2]
plt.imshow(f)
print(f.max())
print(f.min())


In [ ]:
def loss_fn(image, levels=5):
  features = compute_features(params, image)
  loss = 0.
  stuff = defaultdict(list)
  assert len(features) == len(reference_features)
  for i, (feat, feat_ref) in enumerate(zip(features, reference_features)):
    ls = jax.image.resize(log_sigma, feat.shape[1:], "linear", antialias=True)
    ls = jax.nn.relu(ls - jnp.log2(log_sigma.shape[-1] / feat.shape[-1]))
    l, s = wasserstein_distortion(feat, feat_ref, ls, levels=levels)
    loss += l
    if i > 0:
      levels = max(levels - 1, 0)
    for j in s:
      stuff[j].append(s[j])
  return loss, stuff


@jax.jit
def train_step(image, opt_state):
  (loss, stuff), grad = jax.value_and_grad(loss_fn, has_aux=True)(image)
  update, opt_state = optimizer.update(grad, opt_state)
  image = optax.apply_updates(image, update)
  return loss, image, opt_state, stuff


def visualize(image, history, stuff=None):
  colab.output.clear(wait=True)

  imgs = [image, reference, reference - image + .5]
  imgs = [jnp.transpose(i, (1, 2, 0)) for i in imgs]
  mediapy.show_images(imgs)

  if stuff is not None:
    for i in range(len(stuff["weights"])):
      for j in range(len(stuff["weights"][i])):
        mediapy.show_images([
            stuff["weights"][i][j],
            stuff["wd_maps"][i][j][0],
        ], vmin=0, vmax=1)

  plt.figure()
  plt.plot(history["loss"])
  plt.xlabel("epoch")
  plt.ylabel("loss")
  plt.yscale("log")
  plt.grid()

  plt.show()


In [ ]:
%no_vertical_scrollbar

num_epochs = 300
steps_per_epoch = 20

optimizer = optax.adam(learning_rate=1e-2)
image = jax.random.uniform(jax.random.PRNGKey(8762345), shape=reference.shape)
opt_state = optimizer.init(image)
history = defaultdict(list)
visualize(image, history)

for epoch in range(num_epochs):
  results = defaultdict(lambda: 0)
  for _ in range(steps_per_epoch):
    loss, image, opt_state, stuff = train_step(image, opt_state)
    results["loss"] += loss
  for k in results:
    history[k].append(results[k] / steps_per_epoch)
  visualize(image, history, stuff)
